**Thành Viên: Phạm Đức Thể, Trần Triệu Vũ, Võ Minh Trí**

**MSSV: 19522253, 19522539, 19522396**

**Lớp: DS200.M21**

**Nội Dung: [Bài tập Kaggle 1](https://www.kaggle.com/competitions/uit-2021-2022-kaggle-assignment-1)**

**[Dataset](https://drive.google.com/drive/folders/1rMbbd5wsoTQI5YjYDglIQegayRwimYrH?usp=sharing)**

**Ngày: 19/05/2022** 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Fg383c5rLwx5TWuTmC9PPZA0bkqg5UtA?usp=sharing)

# **Big Data - UIT DS200 Kaggle Assignment 5**

## **Import Library**

In [ ]:
!pip install pyspark==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 57.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=6712edd9e557c91e12575ce446f5624b9d7f1d288156138f705c2db8f835674a
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
import numpy as np
import pandas
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pprint import pprint

spark = SparkSession.builder.appName('BigDataAssignment5').getOrCreate()
spark

## **Read Data**

In [ ]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 3 2021-2022/HỌC KỲ 2 2021 - 2022/DS200.M21 - PHÂN TÍCH DỮ LIỆU LỚN/THỰC HÀNH/LAB/Assignment05 - 19 05 2022/Dataset/'

### genome_scores

In [ ]:
genome_scores = spark.read.csv(path=PATH+'genome_scores.csv',
                              sep=',', header=True, quote='"',
                              schema='tagId INT, movieId INT, relevance DOUBLE').cache()

genome_scores.printSchema()
genome_scores.show()
genome_scores.summary().show()

root
 |-- tagId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- relevance: double (nullable = true)

+-----+-------+--------------------+
|tagId|movieId|           relevance|
+-----+-------+--------------------+
|    1|  49910|0.023749999999999993|
|    2|  49910|0.022749999999999992|
|    3|  49910|0.030000000000000027|
|    4|  49910| 0.03199999999999997|
|    5|  49910| 0.13474999999999998|
|    6|  49910| 0.07324999999999998|
|    7|  49910|0.059250000000000025|
|    8|  49910| 0.07650000000000001|
|    9|  49910|             0.03125|
|   10|  49910|0.017249999999999988|
|   11|  49910|0.032999999999999974|
|   12|  49910| 0.01974999999999999|
|   13|  49910|0.058750000000000024|
|   14|  49910|0.014249999999999985|
|   17|  49910|0.010749999999999982|
|   18|  49910|             0.09825|
|   19|  49910|               0.118|
|   20|  49910|             0.09425|
|   21|  49910|              0.3005|
|   22|  49910| 0.30000000000000004|
+-----+-------+---------

### genome_tags

In [ ]:
genome_tags = spark.read.csv(path=PATH+'genome_tags.csv',
                             sep=',', header=True, quote='"',
                             schema='tag STRING, tagId INT').cache()

genome_tags.printSchema()
genome_tags.show()
genome_tags.summary().show()

root
 |-- tag: string (nullable = true)
 |-- tagId: integer (nullable = true)

+------------------+-----+
|               tag|tagId|
+------------------+-----+
|             1970s|    8|
|             anime|   65|
|               art|   75|
|            biting|  137|
|        creativity|  266|
|intelligent sci-fi|  552|
|    love triangles|  615|
|           lyrical|  618|
|             oscar|  745|
|            poetry|  790|
|            travel| 1042|
|            absurd|   18|
|      dark fantasy|  287|
|     depp & burton|  295|
|      life & death|  601|
|             mafia|  622|
|             scifi|  890|
|            sequel|  901|
|        television| 1019|
|              70mm|   12|
+------------------+-----+
only showing top 20 rows

+-------+------------------+-----------------+
|summary|               tag|            tagId|
+-------+------------------+-----------------+
|  count|              1042|             1042|
|   mean|               5.0| 562.485604606526|
| stddev|2.8

### movies

In [ ]:
movies = spark.read.csv(path=PATH+'movies.csv',
                              sep=',', header=True, quote='"',
                              schema='movieId INT, title STRING, genres STRING, year INT').cache()

movies.printSchema()
movies.show()
movies.summary().show()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- year: integer (nullable = true)

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|  49910|     Freedom Writers|           ['Drama']|2007|
|  50149|Happily N'Ever After|['Animation', 'Ch...|2007|
|  50151|   Home of the Brave|['Action', 'Drama...|2007|
|  50153|Code Name: The Cl...|['Action', 'Comed...|2007|
|  50158|      Stomp the Yard|['Drama', 'Musical']|2007|
|  50162|Arthur and the In...|['Action', 'Child...|2007|
|  50440|            Primeval|['Horror', 'Thril...|2007|
|  50442|           Alpha Dog|  ['Crime', 'Drama']|2007|
|  50445|        Hitcher, The|['Action', 'Horro...|2007|
|  50601|Bridge to Terabithia|['Adventure', 'Ch...|2007|
|  50685|            Waitress|['Comedy', 'Drama...|2007|
|  50796| Blood and Chocolate|['Drama', 'Fan

### ratings

In [ ]:
ratings = spark.read.csv(path=PATH+'ratings.csv',
                              sep=',', header=True, quote='"',
                              schema='userId INT, movieId INT, rating DOUBLE, timestamp INT') \
.withColumn('timestamp', f.to_timestamp(f.from_unixtime('timestamp'))).cache()

ratings.printSchema()
ratings.show()
ratings.summary().show()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|   471|  57669|   3.5|2018-01-28 16:04:30|
|   471|  58295|   3.5|2018-01-28 16:12:54|
|   471|  58803|   3.0|2018-01-28 16:00:30|
|   471|  59315|   3.0|2018-01-28 15:55:59|
|   471|  59369|   3.0|2018-01-28 16:19:04|
|   471|  60069|   4.0|2018-01-28 16:04:29|
|   471|  60074|   3.5|2018-01-28 15:49:43|
|   471|  60684|   3.0|2018-01-28 15:58:58|
|   471|  61323|   3.5|2018-01-28 16:04:18|
|   471|  64614|   4.5|2018-01-28 16:04:16|
|   471|  64622|   3.0|2018-01-28 16:22:03|
|   471|  64716|   3.5|2018-01-28 16:12:10|
|   471|  64957|   3.5|2018-01-28 15:58:31|
|   471|  65682|   2.0|2018-01-28 15:58:28|
|   471|  68157|   4.0|2018-01-28 15:52:16|
|   471|  68319|   2.5|2018-01-28 15:49:4

### tags

In [ ]:
tags = spark.read.csv(path=PATH+'tags.csv',
                              sep=',', header=True, quote='"',
                              schema='userId INT, movieId INT, tag STRING, timestamp INT') \
.withColumn('timestamp', f.to_timestamp(f.from_unixtime('timestamp'))).cache()

tags.printSchema()
tags.show()
tags.summary().show()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+--------------------+-------------------+
|userId|movieId|                 tag|          timestamp|
+------+-------+--------------------+-------------------+
|   719| 160980|          flashbacks|2017-04-11 14:18:51|
|   719| 160980|           realistic|2017-04-11 14:18:55|
|   719| 160980|           Tom Hanks|2017-04-11 14:18:59|
|  1089|  77561|         Don Cheadle|2018-08-22 00:49:33|
|  1089|  77561|              Marvel|2018-08-22 00:49:38|
|  1089|  77561|                 MCU|2018-08-22 00:49:42|
|  1089|  77561|       Mickey Rourke|2018-08-22 00:49:31|
|  1089|  77561|   Robert Downey Jr.|2018-08-22 00:49:24|
|  1089|  77561|   Samuel L. Jackson|2018-08-22 00:49:35|
|  1089|  77561|  Scarlett Johansson|2018-08-22 00:49:26|
|  1089|  77561|              sci-fi|2018-08-22 00:49:28|
|  1089|  77561|        

### test

In [ ]:
test = spark.read.csv(path=PATH+'test.csv',
                              sep=',', header=True, quote='"',
                              schema='ratingId INT, userId INT, movieId INT').cache()

test.printSchema()
test.show()
test.summary().show()

root
 |-- ratingId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)

+--------+------+-------+
|ratingId|userId|movieId|
+--------+------+-------+
|       0|   471|  57368|
|       1|   471|  58559|
|       2|   471|  68954|
|       3|   471|  76093|
|       4|   471|  81591|
|       5|   471|  96821|
|       6|   471|  97921|
|       7|   471| 106920|
|       8|   471| 122916|
|       9|   471| 134130|
|      10|   471| 164179|
|      11|  6658|  59615|
|      12|  6658|  76251|
|      13|  6658|  79702|
|      14|  6658|  85342|
|      15|  6658|  86332|
|      16|  6658|  90746|
|      17|  6658|  94959|
|      18|  6658|  96079|
|      19|  6658| 102051|
+--------+------+-------+
only showing top 20 rows

+-------+---------------+------------------+------------------+
|summary|       ratingId|            userId|           movieId|
+-------+---------------+------------------+------------------+
|  count|         228750|       

### sampleSolutions

In [ ]:
sampleSolutions = spark.read.csv(path=PATH+'sampleSolutions.csv',
                              sep=',', header=True, quote='"',
                              schema='ratingId INT, rating DOUBLE').cache()

sampleSolutions.printSchema()
sampleSolutions.show()
sampleSolutions.summary().show()

root
 |-- ratingId: integer (nullable = true)
 |-- rating: double (nullable = true)

+--------+------+
|ratingId|rating|
+--------+------+
|       0|   3.5|
|       1|   4.5|
|       2|   3.5|
|       3|   3.0|
|       4|   5.0|
|       5|   4.5|
|       6|   4.5|
|       7|   0.5|
|       8|   4.5|
|       9|   1.0|
|      10|   1.0|
|      11|   0.5|
|      12|   3.0|
|      13|   1.0|
|      14|   3.5|
|      15|   2.0|
|      16|   3.5|
|      17|   4.5|
|      18|   5.0|
|      19|   1.5|
+--------+------+
only showing top 20 rows

+-------+---------------+------------------+
|summary|       ratingId|            rating|
+-------+---------------+------------------+
|  count|         228750|            228750|
|   mean|       114374.5|2.7478142076502734|
| stddev|66034.581375973|1.4350336968691324|
|    min|              0|               0.5|
|    25%|          57187|               1.5|
|    50%|         114356|               2.5|
|    75%|         171542|               4.0|
|    ma

## **Build Recommendation Model**

In [ ]:
training_data = ratings.select('userId', 'movieId', 'rating')
training_data.show()
training_data.count()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   471|  57669|   3.5|
|   471|  58295|   3.5|
|   471|  58803|   3.0|
|   471|  59315|   3.0|
|   471|  59369|   3.0|
|   471|  60069|   4.0|
|   471|  60074|   3.5|
|   471|  60684|   3.0|
|   471|  61323|   3.5|
|   471|  64614|   4.5|
|   471|  64622|   3.0|
|   471|  64716|   3.5|
|   471|  64957|   3.5|
|   471|  65682|   2.0|
|   471|  68157|   4.0|
|   471|  68319|   2.5|
|   471|  69122|   3.0|
|   471|  69757|   3.5|
|   471|  69844|   3.0|
|   471|  70286|   2.5|
+------+-------+------+
only showing top 20 rows



1859015

In [ ]:
als = ALS(userCol='userId', 
          itemCol='movieId', 
          ratingCol='rating',
          maxIter = 25,
          rank = 200,
          regParam = 0.0497,
          # coldStartStrategy='drop',
          implicitPrefs=False)

In [ ]:
evaluator = RegressionEvaluator(metricName='rmse', 
                                labelCol='rating', 
                                predictionCol='prediction')

In [ ]:
start = time.time()
model = als.fit(training_data)
end = time.time()
print(f'Completed: {(end-start)/60} minute')

Completed: 48.83780017693837 minute


In [ ]:
predictions = model.transform(training_data)
predictions.show()

rmse = evaluator.evaluate(predictions.na.drop())
print(f'RMSE: {rmse}')

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|141903|  57370|   1.0| 1.8362172|
| 60702|  57370|   3.5|  3.467981|
|278022|  57370|   4.0| 3.3394425|
| 37046|  57370|   3.0| 2.8648062|
| 87439|  57370|   4.0|  3.821837|
| 63413|  57370|   4.0| 3.3372738|
|137652|  57370|   3.0| 2.4689443|
|240801|  57370|   2.5| 2.6483066|
|172271|  57370|   3.0| 2.9801223|
|173071|  57370|   3.0| 2.5658326|
|262786|  57370|   3.5|  3.470858|
|180933|  57370|   0.5|  2.172019|
|258878|  57370|   0.5| 1.4940329|
|193493|  57370|   2.0| 1.9941074|
|108517|  57370|   1.5| 1.8325514|
|198851|  57370|   3.0| 2.6974838|
| 57343|  57370|   4.5|  4.061445|
|  3144|  57370|   2.5| 2.9912598|
| 71955|  57370|   3.0|  2.730238|
|161754|  57370|   2.5| 2.4273937|
+------+-------+------+----------+
only showing top 20 rows

RMSE: 0.5309489471585769


## **Prediction**

### Prediction on Test

In [ ]:
pred = model.transform(test).withColumnRenamed("prediction","rating")
pred.show()
pred.count()

+--------+------+-------+------+
|ratingId|userId|movieId|rating|
+--------+------+-------+------+
|  220163| 50223|  54190|   NaN|
|  221700| 92794|  54190|   NaN|
|  221819| 95467|  54190|   NaN|
|  224882|179209|  54190|   NaN|
|  224731|174785|  54190|   NaN|
|  224658|173071|  54190|   NaN|
|  220350| 55668|  54190|   NaN|
|  222674|117633|  54190|   NaN|
|  218638|  6606|  54190|   NaN|
|  223978|153759|  54190|   NaN|
|  218708|  8646|  54190|   NaN|
|  219755| 39145|  54190|   NaN|
|  226467|220799|  54190|   NaN|
|  222976|125606|  54190|   NaN|
|  226212|212863|  54190|   NaN|
|  223980|153781|  54190|   NaN|
|  226067|209147|  54190|   NaN|
|  227523|250921|  54190|   NaN|
|  224699|173949|  54190|   NaN|
|  218483|  2232|  54190|   NaN|
+--------+------+-------+------+
only showing top 20 rows



228750

### Test Join

In [ ]:
genres_pred = pred.join(movies,
                        on = "movieId",
                        how = "left_outer").select("ratingId","userId","movieId","genres","rating")

genres_pred.show(truncate=False)
genres_pred.count()

+--------+------+-------+------------------------------------------+------+
|ratingId|userId|movieId|genres                                    |rating|
+--------+------+-------+------------------------------------------+------+
|220163  |50223 |54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|221700  |92794 |54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|221819  |95467 |54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|224882  |179209|54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|224731  |174785|54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|224658  |173071|54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|220350  |55668 |54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|222674  |117633|54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|218638  |6606  |54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|223978  |153759|54190  |['Drama', 'Fantasy', 'Musical', 'Romance']|NaN   |
|218708  |86

228750

In [ ]:
tag_pred = pred.join(tags,
                     on = ["userId","movieId"],
                     how = "left_outer").select("ratingId","userId","movieId","tag","rating")
tag_pred.show()
tag_pred.count()

+--------+------+-------+----+------+
|ratingId|userId|movieId| tag|rating|
+--------+------+-------+----+------+
|  220163| 50223|  54190|null|   NaN|
|  221700| 92794|  54190|null|   NaN|
|  221819| 95467|  54190|null|   NaN|
|  224882|179209|  54190|null|   NaN|
|  224731|174785|  54190|null|   NaN|
|  224658|173071|  54190|null|   NaN|
|  220350| 55668|  54190|null|   NaN|
|  222674|117633|  54190|null|   NaN|
|  218638|  6606|  54190|null|   NaN|
|  223978|153759|  54190|null|   NaN|
|  218708|  8646|  54190|null|   NaN|
|  219755| 39145|  54190|null|   NaN|
|  226467|220799|  54190|null|   NaN|
|  222976|125606|  54190|null|   NaN|
|  226212|212863|  54190|null|   NaN|
|  223980|153781|  54190|null|   NaN|
|  226067|209147|  54190|null|   NaN|
|  227523|250921|  54190|null|   NaN|
|  224699|173949|  54190|null|   NaN|
|  218483|  2232|  54190|null|   NaN|
+--------+------+-------+----+------+
only showing top 20 rows



247729

### Train Join

In [ ]:
genres_train = training_data.join(movies,
                                  on = "movieId",
                                  how = "left_outer").select("userId","movieId","genres","rating")

genres_train.show()
genres_train.count()

+------+-------+--------------------+------+
|userId|movieId|              genres|rating|
+------+-------+--------------------+------+
|   471|  57669|['Comedy', 'Crime...|   3.5|
|   471|  58295|['Action', 'Crime...|   3.5|
|   471|  58803|['Crime', 'Drama'...|   3.0|
|   471|  59315|['Action', 'Adven...|   3.0|
|   471|  59369|['Action', 'Crime...|   3.0|
|   471|  60069|['Adventure', 'An...|   4.0|
|   471|  60074|['Action', 'Adven...|   3.5|
|   471|  60684|['Action', 'Drama...|   3.0|
|   471|  61323|['Comedy', 'Crime...|   3.5|
|   471|  64614|  ['Crime', 'Drama']|   4.5|
|   471|  64622|['Drama', 'Romance']|   3.0|
|   471|  64716|           ['Drama']|   3.5|
|   471|  64957|['Drama', 'Fantas...|   3.5|
|   471|  65682|['Action', 'Fanta...|   2.0|
|   471|  68157|['Action', 'Drama...|   4.0|
|   471|  68319|['Action', 'Sci-F...|   2.5|
|   471|  69122| ['Comedy', 'Crime']|   3.0|
|   471|  69757|['Comedy', 'Drama...|   3.5|
|   471|  69844|['Adventure', 'Fa...|   3.0|
|   471|  

1859015

In [ ]:
tag_train = training_data.join(tags,
                               ["userId","movieId"],
                               how = "left_outer").select("userId","movieId","tag","rating")

tag_train.show()
tag_train.count()

+------+-------+----+------+
|userId|movieId| tag|rating|
+------+-------+----+------+
|   471|  57669|null|   3.5|
|   471|  58295|null|   3.5|
|   471|  58803|null|   3.0|
|   471|  59315|null|   3.0|
|   471|  59369|null|   3.0|
|   471|  60069|null|   4.0|
|   471|  60074|null|   3.5|
|   471|  60684|null|   3.0|
|   471|  61323|null|   3.5|
|   471|  64614|null|   4.5|
|   471|  64622|null|   3.0|
|   471|  64716|null|   3.5|
|   471|  64957|null|   3.5|
|   471|  65682|null|   2.0|
|   471|  68157|null|   4.0|
|   471|  68319|null|   2.5|
|   471|  69122|null|   3.0|
|   471|  69757|null|   3.5|
|   471|  69844|null|   3.0|
|   471|  70286|null|   2.5|
+------+-------+----+------+
only showing top 20 rows



2023827

In [ ]:
tag_rating = tag_train.filter(f.col("tag").isNotNull()).groupby("tag","movieId").agg(f.mean(f.col("rating")).alias("tag_rating"))

tag_rating.show()
tag_rating.count()

+--------------------+-------+------------------+
|                 tag|movieId|        tag_rating|
+--------------------+-------+------------------+
|     Natalie Portman|  58154| 3.269230769230769|
|   behind-the-scenes| 109153|               4.5|
|           narration| 179337|               3.5|
|     Jake Gyllenhaal|  78105|3.1818181818181817|
|oscar (best suppo...|  99114|               4.5|
|     Nicholas Sparks| 100527|               1.0|
|           superhero| 103772|3.0454545454545454|
|            wormhole| 109487| 4.232142857142857|
|           overrated|  69757|              2.85|
|        Winona Ryder|  81591|             3.875|
|          Simplistic|  89904|               4.0|
|      twists & turns| 101864|3.7758620689655173|
|        Hugh Jackman| 104879| 4.157894736842105|
|  Strong female lead| 182715|3.5526315789473686|
|        great acting|  88125|              4.75|
|   music documentary|  96662|               0.5|
|      cinematography| 112183|             4.125|


78822

In [ ]:
genres_rating = genres_train.withColumn("genres",f.regexp_replace("genres","\[|\]|\'",""))\
.withColumn("genres",f.split("genres",", "))\
.withColumn("genres",f.explode("genres"))\
.groupby("genres","userId").agg(f.mean(f.col("rating")).alias("genres_rating"))

genres_rating.show()
genres_rating.count()

+---------+------+------------------+
|   genres|userId|     genres_rating|
+---------+------+------------------+
|  Romance| 19553|3.3333333333333335|
|Adventure| 40335|3.3620689655172415|
| Children| 64822|3.5833333333333335|
|  Romance|131213|              3.15|
|  Fantasy|245857|         3.3515625|
|    Drama| 21598| 3.267605633802817|
|     IMAX| 36655| 3.484848484848485|
|  Mystery| 65905| 4.038461538461538|
|Film-Noir| 86672|               3.0|
|Animation|154927|               3.5|
|    Crime|213468| 4.333333333333333|
|  Fantasy|282203| 2.847826086956522|
|    Crime|112032| 3.336734693877551|
|  Musical|  2025|3.8666666666666667|
|Film-Noir|  8911|              2.25|
|  Mystery| 48623|              3.25|
|      War|117375|               4.5|
| Children|251641| 4.607142857142857|
|   Sci-Fi|102780| 3.760869565217391|
| Children|262754|              3.25|
+---------+------+------------------+
only showing top 20 rows



161235

### Train Join Test

Nối tag_rating_pred với movie tạo genres_pred

In [ ]:
#Tach genres nhỏ ra
## Đầu tiên phải fill nan ở genres để genres split không mất dữ liệu
genres_pred = genres_pred.na.fill("NaN")

genres_pred.show()
genres_pred.count()

+--------+------+-------+--------------------+------+
|ratingId|userId|movieId|              genres|rating|
+--------+------+-------+--------------------+------+
|  220163| 50223|  54190|['Drama', 'Fantas...|   NaN|
|  221700| 92794|  54190|['Drama', 'Fantas...|   NaN|
|  221819| 95467|  54190|['Drama', 'Fantas...|   NaN|
|  224882|179209|  54190|['Drama', 'Fantas...|   NaN|
|  224731|174785|  54190|['Drama', 'Fantas...|   NaN|
|  224658|173071|  54190|['Drama', 'Fantas...|   NaN|
|  220350| 55668|  54190|['Drama', 'Fantas...|   NaN|
|  222674|117633|  54190|['Drama', 'Fantas...|   NaN|
|  218638|  6606|  54190|['Drama', 'Fantas...|   NaN|
|  223978|153759|  54190|['Drama', 'Fantas...|   NaN|
|  218708|  8646|  54190|['Drama', 'Fantas...|   NaN|
|  219755| 39145|  54190|['Drama', 'Fantas...|   NaN|
|  226467|220799|  54190|['Drama', 'Fantas...|   NaN|
|  222976|125606|  54190|['Drama', 'Fantas...|   NaN|
|  226212|212863|  54190|['Drama', 'Fantas...|   NaN|
|  223980|153781|  54190|['D

228750

In [ ]:
genres_pred = genres_pred.withColumn("genres",f.regexp_replace("genres","\[|\]|\'","")) \
.withColumn("genres",f.split("genres",", "))\
.withColumn("genres",f.explode("genres"))\

genres_pred.show()
genres_pred.count()

+--------+------+-------+-------+------+
|ratingId|userId|movieId| genres|rating|
+--------+------+-------+-------+------+
|  220163| 50223|  54190|  Drama|   NaN|
|  220163| 50223|  54190|Fantasy|   NaN|
|  220163| 50223|  54190|Musical|   NaN|
|  220163| 50223|  54190|Romance|   NaN|
|  221700| 92794|  54190|  Drama|   NaN|
|  221700| 92794|  54190|Fantasy|   NaN|
|  221700| 92794|  54190|Musical|   NaN|
|  221700| 92794|  54190|Romance|   NaN|
|  221819| 95467|  54190|  Drama|   NaN|
|  221819| 95467|  54190|Fantasy|   NaN|
|  221819| 95467|  54190|Musical|   NaN|
|  221819| 95467|  54190|Romance|   NaN|
|  224882|179209|  54190|  Drama|   NaN|
|  224882|179209|  54190|Fantasy|   NaN|
|  224882|179209|  54190|Musical|   NaN|
|  224882|179209|  54190|Romance|   NaN|
|  224731|174785|  54190|  Drama|   NaN|
|  224731|174785|  54190|Fantasy|   NaN|
|  224731|174785|  54190|Musical|   NaN|
|  224731|174785|  54190|Romance|   NaN|
+--------+------+-------+-------+------+
only showing top

645703

Thực hiện fill "NaN" cho rating giống ở Tag

In [ ]:
genres_rating_pred = genres_pred.join(genres_rating,
                                      on = ["genres","userId"],
                                      how = "left_outer")

genres_rating_pred.show()
genres_rating_pred.count()

+------------------+------+--------+-------+---------+------------------+
|            genres|userId|ratingId|movieId|   rating|     genres_rating|
+------------------+------+--------+-------+---------+------------------+
|(no genres listed)| 23389|  187587| 147750|3.5815082|3.8333333333333335|
|(no genres listed)| 67220|   48078| 183909|3.2660663|               2.5|
|(no genres listed)| 81924|  146429| 180543|1.2763509|1.7142857142857142|
|            Action|  3631|    5664| 112171|2.5438163|2.8658536585365852|
|            Action|  3631|    5659|  91976|2.4974258|2.8658536585365852|
|            Action|  3631|    5646|  62849|3.0459816|2.8658536585365852|
|            Action|  3631|    5654|  87192|2.5735466|2.8658536585365852|
|            Action|  3631|    5649|  73268|2.3232372|2.8658536585365852|
|            Action|  9328|  209049|  71535|      NaN|              null|
|            Action| 22192|  207108| 122904|      NaN|              null|
|            Action| 25435|  174634|  

645703

In [ ]:
genres_rating_pred = genres_rating_pred.withColumn("rating",
                                                   f.when((f.col("rating") == "NaN")&(f.col("genres_rating").isNotNull()),
                                                          genres_rating_pred.genres_rating).otherwise(genres_rating_pred["rating"]))

genres_rating_pred.show()
genres_rating_pred.count()

+------------------+------+--------+-------+------------------+------------------+
|            genres|userId|ratingId|movieId|            rating|     genres_rating|
+------------------+------+--------+-------+------------------+------------------+
|(no genres listed)| 23389|  187587| 147750| 3.581508159637451|3.8333333333333335|
|(no genres listed)| 67220|   48078| 183909| 3.266066312789917|               2.5|
|(no genres listed)| 81924|  146429| 180543|1.2763508558273315|1.7142857142857142|
|            Action|  3631|    5664| 112171| 2.543816328048706|2.8658536585365852|
|            Action|  3631|    5659|  91976|2.4974257946014404|2.8658536585365852|
|            Action|  3631|    5646|  62849|3.0459816455841064|2.8658536585365852|
|            Action|  3631|    5654|  87192|2.5735466480255127|2.8658536585365852|
|            Action|  3631|    5649|  73268| 2.323237180709839|2.8658536585365852|
|            Action|  9328|  209049|  71535|               NaN|              null|
|   

645703

Tag
(1 userId cùng xem 1 movie có thể có 2 tag)

In [ ]:
tag_pred = genres_rating_pred.join(tags,
                                   on = ["userId","movieId"],
                                   how = "left_outer").select("ratingId","userId","movieId","tag","rating")
tag_pred.show()
tag_pred.count()

+--------+------+-------+----+------------------+
|ratingId|userId|movieId| tag|            rating|
+--------+------+-------+----+------------------+
|  187587| 23389| 147750|null| 3.581508159637451|
|   48078| 67220| 183909|null| 3.266066312789917|
|  146429| 81924| 180543|null|1.2763508558273315|
|    5664|  3631| 112171|null| 2.543816328048706|
|    5659|  3631|  91976|null|2.4974257946014404|
|    5646|  3631|  62849|null|3.0459816455841064|
|    5654|  3631|  87192|null|2.5735466480255127|
|    5649|  3631|  73268|null| 2.323237180709839|
|  209049|  9328|  71535|null|               NaN|
|  207108| 22192| 122904|null|               NaN|
|  174634| 25435|  91529|null|3.0075716972351074|
|  209525| 25556| 111360|null|               NaN|
|  211333| 26613| 108932|null|               NaN|
|  219411| 29635|  51077|null|               NaN|
|   54141| 33549| 122882|null|3.5553195476531982|
|   54138| 33549|  91529|null|3.7499794960021973|
|   54143| 33549| 143355|null| 2.740993022918701|


699110

In [ ]:
tag_rating_pred = tag_pred.join(tag_rating,
                                on = ["tag","movieId"],
                                how = "left_outer")
tag_rating_pred.show()
tag_rating_pred.count()

+----+-------+--------+------+------------------+----------+
| tag|movieId|ratingId|userId|            rating|tag_rating|
+----+-------+--------+------+------------------+----------+
|null|  54190|  226552|223046|               NaN|      null|
|null|  54190|  222674|117633| 4.466666666666667|      null|
|null|  54190|  220350| 55668|               NaN|      null|
|null|  54190|  228230|270101|               NaN|      null|
|null|  54190|  219520| 32492|               NaN|      null|
|null|  54190|  226446|220102|               NaN|      null|
|null|  54190|  222903|123537|3.6785714285714284|      null|
|null|  54190|  226152|211432|               NaN|      null|
|null|  54190|  226101|209976|               NaN|      null|
|null|  54190|  227523|250921|               NaN|      null|
|null|  54190|  218491|  2440| 3.388888888888889|      null|
|null|  54190|  223137|129869|               NaN|      null|
|null|  54190|  226446|220102|               NaN|      null|
|null|  54190|  224882|1

699110

In [ ]:
tag_rating_pred = tag_rating_pred.withColumn("rating",
                                             f.when((f.col("rating") == "NaN")&(f.col("tag_rating").isNotNull()),
                                                    tag_rating_pred.tag_rating).otherwise(tag_rating_pred["rating"]))
tag_rating_pred.show()
tag_rating_pred.count()

+----+-------+--------+------+------------------+----------+
| tag|movieId|ratingId|userId|            rating|tag_rating|
+----+-------+--------+------+------------------+----------+
|null|  54190|  226552|223046|               NaN|      null|
|null|  54190|  222674|117633| 4.466666666666667|      null|
|null|  54190|  220350| 55668|               NaN|      null|
|null|  54190|  228230|270101|               NaN|      null|
|null|  54190|  219520| 32492|               NaN|      null|
|null|  54190|  226446|220102|               NaN|      null|
|null|  54190|  222903|123537|3.6785714285714284|      null|
|null|  54190|  226152|211432|               NaN|      null|
|null|  54190|  226101|209976|               NaN|      null|
|null|  54190|  227523|250921|               NaN|      null|
|null|  54190|  218491|  2440| 3.388888888888889|      null|
|null|  54190|  223137|129869|               NaN|      null|
|null|  54190|  226446|220102|               NaN|      null|
|null|  54190|  224882|1

699110

In [ ]:
#Tinh trung binh rating theo tung nhom userId
user_rating = training_data.groupby("userId").agg(f.mean(f.col("rating")).alias("user_rating"))
user_rating.show()
user_rating.count()

+------+------------------+
|userId|       user_rating|
+------+------------------+
|   471| 3.276595744680851|
|  6658|3.4692307692307693|
| 19553|3.6908212560386473|
| 28088|3.5485436893203883|
| 40335| 3.141509433962264|
| 41946|3.0422535211267605|
| 42834|             3.536|
| 48254|3.3812154696132595|
| 53634|            3.6125|
| 53691|             3.465|
| 54190| 3.566744730679157|
| 57984|         4.3515625|
| 64121|              3.47|
| 64822| 3.934285714285714|
| 70097|2.7945205479452055|
| 74166| 3.986842105263158|
| 78113| 2.974264705882353|
| 95940| 3.327294685990338|
|120988|3.2882882882882885|
|121763| 1.962686567164179|
+------+------------------+
only showing top 20 rows



8870

In [ ]:
user_rating_pred = tag_rating_pred.join(user_rating,on = "userId",how = "left_outer")
user_rating_pred.show()
user_rating_pred.count()

+------+----+-------+--------+------------------+----------+-----------------+
|userId| tag|movieId|ratingId|            rating|tag_rating|      user_rating|
+------+----+-------+--------+------------------+----------+-----------------+
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null| 106920|       7| 3.145341157913208|      null|3.276595744680851|
|   471|null| 106920|       7| 3.145341157913208|   

699110

In [ ]:
user_rating_pred = user_rating_pred.withColumn("rating",
                                               f.when((f.col("rating") == "NaN")&(f.col("user_rating").isNotNull()),
                                                      user_rating_pred.user_rating).otherwise(user_rating_pred["rating"]))
user_rating_pred.show()
user_rating_pred.count()

+------+----+-------+--------+------------------+----------+-----------------+
|userId| tag|movieId|ratingId|            rating|tag_rating|      user_rating|
+------+----+-------+--------+------------------+----------+-----------------+
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  68954|       2| 3.521733522415161|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null|  58559|       1|3.5132784843444824|      null|3.276595744680851|
|   471|null| 106920|       7| 3.145341157913208|      null|3.276595744680851|
|   471|null| 106920|       7| 3.145341157913208|   

699110

In [ ]:
#Tinh trung binh rating theo tung nhom movieId
movie_rating = training_data.groupby("movieId").agg(f.mean(f.col("rating")).alias("movie_rating"))
movie_rating.show()
movie_rating.count()

+-------+------------------+
|movieId|      movie_rating|
+-------+------------------+
|  96488| 3.934192672998643|
|  68135|2.9404104183109707|
| 139747|3.5134228187919465|
| 160563|3.0233208955223883|
|  96853|2.8333333333333335|
| 175197| 2.701570680628272|
|  76143| 3.413793103448276|
| 176073|3.1923076923076925|
| 141422| 3.388535031847134|
|  57370|2.7837837837837838|
| 166558|3.1666666666666665|
| 109800|3.5416666666666665|
| 156017|3.2222222222222223|
| 133153|               2.4|
| 124861| 3.840909090909091|
|  96469|3.3846153846153846|
| 162260|               4.0|
| 162296|               2.0|
|  97186|3.6578947368421053|
|  99817|               3.7|
+-------+------------------+
only showing top 20 rows



19332

In [ ]:
movie_rating_pred = user_rating_pred.join(movie_rating,on = "movieId",how = "left_outer")
movie_rating_pred.show()
movie_rating_pred.count()

+-------+------+----+--------+------------------+----------+------------------+------------+
|movieId|userId| tag|ratingId|            rating|tag_rating|       user_rating|movie_rating|
+-------+------+----+--------+------------------+----------+------------------+------------+
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 92794|null|  221700|2.9444444444444446|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700|             2.625|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700| 2.888059701492537|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700|               3.0|      null|2.74045801

699110

In [ ]:
movie_rating_pred = movie_rating_pred.withColumn("rating",
                                                 f.when((f.col("rating") == "NaN")&(f.col("movie_rating").isNotNull()),
                                                        movie_rating_pred.movie_rating).otherwise(movie_rating_pred["rating"]))
movie_rating_pred.show()
movie_rating_pred.count()

+-------+------+----+--------+------------------+----------+------------------+------------+
|movieId|userId| tag|ratingId|            rating|tag_rating|       user_rating|movie_rating|
+-------+------+----+--------+------------------+----------+------------------+------------+
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 50223|null|  220163|               NaN|      null|              null|        null|
|  54190| 92794|null|  221700|2.9444444444444446|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700|             2.625|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700| 2.888059701492537|      null|2.7404580152671754|        null|
|  54190| 92794|null|  221700|               3.0|      null|2.74045801

699110

In [ ]:
final = movie_rating_pred.groupby("ratingId").agg(f.mean(f.col("rating")).alias("rating"))
final.show()
final.count()

+--------+------------------+
|ratingId|            rating|
+--------+------------------+
|   31035| 2.546381711959839|
|   55155| 4.131705284118652|
|   97419| 3.672532081604004|
|   10817|3.2172927856445312|
|   87120| 3.880873203277588|
|   15619|2.4686973094940186|
|   59355|2.5683367252349854|
|  197890|1.7258273363113403|
|  126373|3.5900206565856934|
|  156017|3.5659642219543457|
|   29285| 2.999251127243042|
|  103357|0.8804675340652466|
|  175197| 3.885437250137329|
|  138920|2.9416871070861816|
|  227152|3.6265120967741935|
|    3175| 3.464491605758667|
|   31528| 3.300706624984741|
|   94950| 2.678312301635742|
|  195367| 2.931645154953003|
|   20924|1.9416104555130005|
+--------+------------------+
only showing top 20 rows



228750

### Final Result

In [ ]:
mean = training_data.groupby("userId").agg(f.mean(f.col("rating")).alias("rating")).agg({"rating":"mean"}).collect()[0]["avg(rating)"]
print(mean)

3.4625665115147357


In [ ]:
random = np.random.choice(np.arange(mean-1,mean+1,0.5))
print(random)

2.9625665115147357


In [ ]:
final = final.withColumn("rating",
                         f.when(final["rating"] < 0, 0.0).when(f.col("rating") == "NaN",mean).otherwise(final["rating"]))
final.show()
final.count()

+--------+------------------+
|ratingId|            rating|
+--------+------------------+
|   31035| 2.546381711959839|
|   55155| 4.131705284118652|
|   97419| 3.672532081604004|
|   10817|3.2172927856445312|
|   87120| 3.880873203277588|
|   15619|2.4686973094940186|
|   59355|2.5683367252349854|
|  197890|1.7258273363113403|
|  126373|3.5900206565856934|
|  156017|3.5659642219543457|
|   29285| 2.999251127243042|
|  103357|0.8804675340652466|
|  175197| 3.885437250137329|
|  138920|2.9416871070861816|
|  227152|3.6265120967741935|
|    3175| 3.464491605758667|
|   31528| 3.300706624984741|
|   94950| 2.678312301635742|
|  195367| 2.931645154953003|
|   20924|1.9416104555130005|
+--------+------------------+
only showing top 20 rows



228750

In [ ]:
final_df = final.toPandas()
def Scale(df):
    for i in range(df.shape[0]):
        j = df.iloc[i,1]
        k = int(j)
        if (j-k) < 0.25:
            df.iloc[i,1] = k
        else:
            if (j-k) < 0.75:
                df.iloc[i,1] = k + 0.5
            else:
                df.iloc[i,1] = k + 1
    return df
#final_df = Scale(final_df)
final_df.loc[final_df.rating > 5.0,"rating"] = 5.0

In [ ]:
final_df.to_csv("submission.csv",index = 0, header = True)

In [ ]:
final_df

,ratingId,rating
0,31035,2.546382
1,55155,4.131705
2,97419,3.672532
3,10817,3.217293
4,87120,3.880873
...,...,...
228745,195486,3.877161
228746,227123,3.462567
228747,82180,3.416496
228748,143190,4.332587
